# Visualizing embeddings in 3D

The example uses [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the dimensionality fo the embeddings from 1536 to 3. Then we can visualize the data points in a 3D plot. The small dataset `dbpedia_samples.jsonl` is curated by randomly sampling 200 samples from [DBpedia validation dataset](https://www.kaggle.com/danofer/dbpedia-classes?select=DBPEDIA_val.csv).

### 1. Load the dataset and query embeddings

In [46]:
%pip install ipympl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [47]:
%pip install tiktoken openai pandas matplotlib plotly scikit-learn numpy python-dotenv matplotlib widget

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement widget (from versions: none)
ERROR: No matching distribution found for widget
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [48]:
from dotenv import load_dotenv
import os

load_dotenv()  


import openai
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [49]:
from openai import OpenAI
client = OpenAI()

client.models.list()


SyncPage[Model](data=[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system')

In [50]:
from openai import OpenAI
client = OpenAI()

client.models.list()

client.models.retrieve("gpt-3.5-turbo-instruct")

client.models.retrieve("gpt-4o-2024-05-13") # the latest models

client.models.retrieve("gpt-4o")

Model(id='gpt-4o', created=1715367049, object='model', owned_by='system')

In [51]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input ="Biden repeats false claim that inflation was at 9% when he took office after being called out last week",
    model = "text-embedding-3-small"
)

print (response.data[0].embedding)

[-0.04104156792163849, -0.013272417709231377, 0.0024851926136761904, 0.025353027507662773, 0.009843258187174797, 0.029253492131829262, -0.014745927415788174, 0.052092887461185455, -0.006760807242244482, -0.00011469468154245988, 0.00949655007570982, -0.024702949449419975, -0.0228827316313982, -0.010634186677634716, 0.0003880150616168976, 0.026761528104543686, 0.00514915632084012, -0.00957780983299017, -0.018711401149630547, -0.021604245528578758, -0.04236339032649994, 0.02758496068418026, -0.044595323503017426, -0.016631152480840683, 0.008824803866446018, -0.011376358568668365, -0.0075029791332781315, -0.028668422251939774, -0.017552096396684647, 0.023987863212823868, 0.003296435112133622, -0.015818554908037186, 0.01780129224061966, -0.019047273322939873, -0.01240564789623022, 0.014323377050459385, 0.009615731425583363, -0.017974646762013435, 0.0722886323928833, -0.018115496262907982, 0.031333740800619125, 0.024962980300188065, -0.07003502547740936, -0.04364187642931938, 0.0513127930462

In [52]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input = "Biden repeats false claim that inflation was at 9% when he took office after being called out last week",
    model = "text-embedding-ada-002"
)
print (response.data[0].embedding)

[-0.038446929305791855, -0.009367747232317924, 0.002974372124299407, 0.007216825149953365, -0.010716086253523827, -0.00899534858763218, -0.03834420070052147, -0.012783540412783623, -0.013650329783558846, 0.013765901327133179, 0.0347486287355423, 0.01729726605117321, 0.022318225353956223, -0.0027913833037018776, 0.005297047086060047, -0.020661693066358566, 0.01587187871336937, 0.004587563686072826, -0.00274162320420146, -0.02724929340183735, -0.019236305728554726, 0.009675939567387104, -0.010022655129432678, 0.013598964549601078, 6.330372707452625e-05, -0.021509220823645592, 0.02172752283513546, -0.008924721740186214, 5.1415463531157e-05, -0.012700071558356285, -0.02459113858640194, -0.036674827337265015, -0.019441766664385796, -0.0005806686822324991, -0.04045017808675766, -0.03400383144617081, -0.009123762138187885, -0.03515955060720444, 0.022703465074300766, 0.0009879794670268893, 0.02343541942536831, 0.006748116575181484, 0.00899534858763218, 0.01680929586291313, -0.0292782224714756,

In [60]:
import pandas as pd
import tiktoken

from utils.embeddings_utils import get_embedding

embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

input_datapath = "data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...


In [61]:
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

/Users/prigogine/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1000